In [ ]:
!poetry --no-ansi add ultralytics requests ipykernel

### Imports

In [ ]:
import ultralytics as ul
import requests
import zipfile
import os, glob, random, shutil
from IPython.display import Image

In [ ]:
from ultralytics import settings

current_dir = os.getcwd()

settings.update(
    {
        'runs_dir' : os.path.join(current_dir, "runs"),
        'weights_dir' : os.path.join(current_dir, "weights"),
    }
)

print(settings)


### Prepare dataset

In [ ]:
# Define dataset URL and local file name
url = "https://www.kaggle.com/api/v1/datasets/download/valentynsichkar/traffic-signs-dataset-in-yolo-format"
zip_path = "traffic-signs-dataset.zip"
extract_folder = "traffic-signs-dataset-original"

# Download the dataset
response = requests.get(url, stream=True)
total_size = int(response.headers.get('content-length', 0))
downloaded_size = 0

if not os.path.exists(extract_folder):
    with open(zip_path, "wb") as file:
        for chunk in response.iter_content(chunk_size=1024*1024):
            if chunk:
                file.write(chunk)
                downloaded_size += len(chunk)
                percentage = (downloaded_size / total_size) * 100
                print(f"Download progress: {percentage:.2f}%")

    # Extract the dataset
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extract_folder)

    # Remove the zip file
    os.remove(zip_path)

    print("Download and extraction completed.")

    with open(os.path.join(extract_folder, ".gitignore"), "w") as file:
        file.write("*")

else:
    print("Dataset already downloaded.")


In [ ]:
image_files = glob.glob(f"{extract_folder}/ts/**/*.jpg", recursive=True)

print(f"Total number of images: {len(image_files)}")

In [ ]:
dataset_folder = "../../datasets/traffic-signs-dataset"
train_size = 0.2

train_folder = os.path.join(dataset_folder, "train")
test_folder = os.path.join(dataset_folder, "test")

train = random.sample(image_files, int(len(image_files) * train_size))
test = list(set(image_files) - set(train))

os.makedirs(os.path.join(train_folder, "images"), exist_ok=True)
os.makedirs(os.path.join(train_folder, "labels"), exist_ok=True)

for file in train:
    label_file = file.replace(".jpg", ".txt")
    shutil.copy(file, os.path.join(train_folder, 'images', file.split("/")[-1]))
    shutil.copy(label_file, os.path.join(train_folder, 'labels', label_file.split("/")[-1]))


os.makedirs(os.path.join(test_folder, "images"), exist_ok=True)
os.makedirs(os.path.join(test_folder, "labels"), exist_ok=True)
for file in test:
    label_file = file.replace(".jpg", ".txt")
    shutil.copy(file, os.path.join(test_folder, 'images', file.split("/")[-1]))
    shutil.copy(label_file, os.path.join(test_folder, 'labels', label_file.split("/")[-1]))


In [ ]:
with open(os.path.join(dataset_folder, "dataset.yaml"), 'w') as file:
    file.write(f"train: train\n")
    file.write(f"val: test\n")
    with open(os.path.join(extract_folder, "classes.names"), 'r') as classes_file:
        classes = classes_file.readlines()
        file.write(f"nc: {len(classes)}\n")
        file.write(f"names:\n")
        for class_name in classes:
            file.write(f"  - {class_name}")
            
with open(os.path.join(dataset_folder, ".gitignore"), 'w') as file:
    file.write("*")

In [ ]:
dataset_yaml_path = f"{os.path.abspath(dataset_folder)}/dataset.yaml"

### Model definition

In [ ]:
model = ul.YOLO("yolo11n.pt")

### Initial tests

In [ ]:
results = model.track(
    'traffic-signs-dataset-original/traffic-sign-to-test.mp4',
    save=True, save_txt=True, save_conf=True, stream=True, 
    )

for _ in results:
    pass


### Training

In [ ]:
dataset_yaml_path = f"{os.path.abspath(dataset_folder)}/dataset.yaml"
print(dataset_yaml_path)
results = model.train(data=dataset_yaml_path, epochs=100, batch=-1, patience=10)

In [ ]:
shutil.copy(f'{os.path.abspath(results.save_dir)}/weights/best.pt', 'finetuned.pt')

### Testing with first video

In [ ]:
finetuned = ul.YOLO("finetuned.pt")

finetuned.track(
    'traffic-signs-dataset-original/traffic-sign-to-test.mp4',
    save=True, save_txt=True, save_conf=True
    )

In [ ]:
# with zipfile.ZipFile("runs.zip", "w", zipfile.ZIP_DEFLATED) as zipf:
#     for root, dirs, files in os.walk("runs"):
#         for file in files:
#             zipf.write(os.path.join(root, file))